In [0]:
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60302712.dfs.core.windows.net",
    "00u5OPJ3GYKrSACO9uvWxs8ZikHiJ7FosLxuXngxN1GqYBs5f2SLr/V0yESwhmfXXSovGYabn6Ai+AStqwR5Tg=="
)


In [0]:
books = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60302712.dfs.core.windows.net/processed/books/"
)
authors = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60302712.dfs.core.windows.net/processed/authors/"
)
reviews = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60302712.dfs.core.windows.net/processed/reviews/"
)

books.show(5)
authors.show(5)

reviews.printSchema()
books.printSchema()
authors.printSchema()


+----------+------------------+------------+-------------+-----+-------+----------+--------+--------------+-----------+--------------------+---------+--------------------+---------+----+--------------------+---------+---------------+-------------+-----------------+-------------------+----------------+--------------------+--------------------+-------+-------------+-------+--------------------+--------------------+
|      isbn|text_reviews_count|country_code|language_code|count|   name|      asin|is_ebook|average_rating|kindle_asin|         description|   format|                link|author_id|role|           publisher|num_pages|publication_day|       isbn13|publication_month|edition_information|publication_year|                 url|           image_url|book_id|ratings_count|work_id|               title|title_without_series|
+----------+------------------+------------+-------------+-----+-------+----------+--------+--------------+-----------+--------------------+---------+----------------

In [0]:
spark.catalog.clearCache()
spark.sql("CLEAR CACHE")
spark.catalog.dropTempView("reviews")


False

In [0]:
reviews = (
    spark.read
    .option("mergeSchema", "true")
    .parquet("abfss://lakehouse@goodreadsreviews60302712.dfs.core.windows.net/processed/reviews/")
)
reviews.count()   # forces scan → rebuilds file list


14971371

In [0]:
from pyspark.sql.functions import col, trim

# Load reviews dataset (silver layer)
reviews = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60302712.dfs.core.windows.net/processed/reviews/"
)

# Preview data and schema
reviews.show(5, truncate=False)
reviews.printSchema()

# Profile for missing or empty values
total_rows = reviews.count()
null_review_id = reviews.filter(col("review_id").isNull()).count()
null_book_id = reviews.filter(col("book_id").isNull()).count()
null_user_id = reviews.filter(col("user_id").isNull()).count()
null_rating = reviews.filter(col("rating").isNull()).count()
empty_text = reviews.filter(
    col("review_text").isNull() | (trim(col("review_text")) == "")
).count()

print(f"Total rows: {total_rows}")
print(f"NULL review_id: {null_review_id}, NULL book_id: {null_book_id}, "
      f"NULL user_id: {null_user_id}, NULL rating: {null_rating}")
print(f"Empty/NULL review_text: {empty_text}")


+--------------------------------+--------+--------------------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.sql.functions import col, trim, length

# Use your loaded reviews DataFrame
df = reviews

#1 Drop rows missing critical keys
df = df.filter(
    col("review_id").isNotNull() &
    col("book_id").isNotNull() &
    col("user_id").isNotNull()
)

#2 Enforce rating to be integer in [1..5]
df = df.withColumn("rating_int", col("rating").cast("int"))
df = df.filter(
    col("rating_int").isNotNull() &
    (col("rating_int") >= 1) &
    (col("rating_int") <= 5)
)

#3 Trim and drop empty/short reviews (<10 chars)
df = df.withColumn("review_text", trim(col("review_text")))
df = df.filter(
    col("review_text").isNotNull() &
    (length(col("review_text")) >= 10)
)

#4 Remove duplicates
df = df.dropDuplicates(["review_id"])

reviews_clean = df.select(
    "review_id",
    "book_id",
    "user_id",
    col("rating_int").alias("rating"),
    "review_text",
    "n_votes",
    "date_added"
)



In [0]:
# Write cleaned reviews to the same silver path
reviews_clean.write.mode("overwrite").parquet(
    "abfss://lakehouse@goodreadsreviews60302712.dfs.core.windows.net/processed/reviews/"
)

# Re-read for validation
reviews_verified = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60302712.dfs.core.windows.net/processed/reviews/"
)

reviews_verified.printSchema()
reviews_verified.show(5, truncate=False)
print(f"Written cleaned rows: {reviews_verified.count()}")



root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- date_added: string (nullable = true)

+--------------------------------+--------+--------------------------------+------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# HomeWork Part 1

from pyspark.sql.functions import col

# Assign aliases to DataFrames for easier column reference
b = books.alias("b")
a = authors.alias("a")
r = reviews_verified.alias("r")

# Join the dataframes
ba = b.join(a, on='author_id', how='inner')
joined = r.join(ba, on='book_id', how='inner')
joined = r.join(ba, on='book_id', how='inner')

# curated reviews gold 
curated_reviews_gold = joined.select(
    col("r.review_id"),
    col("r.book_id"),
    col("b.title"),
    col("a.author_id"),
    col("a.name"),
    col("r.user_id"),
    col("r.rating"),
    col("r.review_text"),
    col("b.language_code"),
    col("r.n_votes"),
    col("r.date_added")
)

# Print the schema and display a few rows to verify the result
print("Curated Reviews Gold DataFrame Schema:")
curated_reviews_gold.printSchema()
print("Curated Reviews Gold DataFrame Sample Rows:")
curated_reviews_gold.show(5, truncate=False)

Curated Reviews Gold DataFrame Schema:
root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- date_added: string (nullable = true)

Curated Reviews Gold DataFrame Sample Rows:
+--------------------------------+--------+-----------------------------------------------------------------------------------+---------+--------------------------+--------------------------------+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# Use a catalog/schema you can write to
#spark.sql("USE CATALOG main")
spark.sql("USE SCHEMA default")

# Persist as a managed Delta table
curated_reviews_gold.write.format("delta").mode("overwrite").saveAsTable("default.curated_reviews")

# Verify
spark.sql("SELECT COUNT(*) AS rows FROM default.curated_reviews").show()
spark.sql("""
  SELECT title, name, rating, language_code
  FROM default.curated_reviews
  ORDER BY date_added DESC
  LIMIT 10
""").show(truncate=False)

+--------+
|    rows|
+--------+
|14970953|
+--------+

+----------------------------------------------------+----------------+------+-------------+
|title                                               |name            |rating|language_code|
+----------------------------------------------------+----------------+------+-------------+
|White Trash Zombie Gone Wild (White Trash Zombie #5)|Diana Rowland   |4     |eng          |
|Böser kleiner Junge                                 |Stephen King    |4     |ger          |
|Sacked (Gridiron, #1)                               |Jen Frederick   |5     |en-CA        |
|Ein ganzes Leben                                    |Robert Seethaler|3     |ger          |
|The Good Girls (The Perfectionists, #2)             |Sara Shepard    |3     |             |
|The Spy Wore Blue (Lord and Lady Spy, #1.5)         |Shana Galen     |5     |eng          |
|Red Queen (Red Queen, #1)                           |Victoria Aveyard|5     |eng          |
|Allegiant (Di